In [1]:
import bs4
import bs4.builder._lxml
import urllib
import pandas as pd
import numpy as np
import requests
import urllib.request, json 
import re
import time
from datetime import date

import hashlib

def hashme(w):
    h = hashlib.md5(w.encode('utf-8'))
    return h.hexdigest()
    
import requests

from urllib.request import Request, urlopen
from urllib.parse import quote

#!pip install PyPDF2
import PyPDF2

from io import StringIO # https://stackoverflow.com/a/18284900
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage

def convert(fname, pages=None):
    if not pages:
        pagenums = set()
    else:
        pagenums = set(pages)

    output = StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)

    infile = open(fname, 'rb') # python three change
    for page in PDFPage.get_pages(infile, pagenums, check_extractable=False):
        interpreter.process_page(page)
    infile.close()
    converter.close()
    text = output.getvalue()
    output.close
    return text
    
def download_pdf(url,path,filename):
    try:
        
        filename = path+filename

        f = requests.get(url, stream=True, headers={'User-agent': 'Mozilla/5.0'})

        with open(filename, "wb") as code:
            #code.write(f.read())
            code.write(f.content)

        return convert(filename)
    except:
        return "error"

def connect_to_page(url):
    connection = requests.get(url)
    html = connection.text
    soup = bs4.BeautifulSoup(html,"lxml")
    connection.close()
    return soup
def download_pdf_unmasked(url,path,filename):
    
        
    filename = path+filename

    f = requests.get(url, stream=True, headers={'User-agent': 'Mozilla/5.0'})

    with open(filename, "wb") as code:
        #code.write(f.read())
        code.write(f.content)

    return convert(filename)

    
def download_docx(book_link, book_name):
    the_book = requests.get(book_link, stream=True)
    with open(book_name, 'wb') as f:
      for chunk in the_book.iter_content(1024 * 1024 * 2):  # 2 MB chunks
        f.write(chunk)

In [ ]:
BASE = "https://www.wicourts.gov"
source = "https://www.wicourts.gov/forms1/circuit/index.htm"





soup = connect_to_page(source)

a_tags = soup.findAll('a',href=True)
count = 0
cat_refs = []
cats_names = []
cat_page_count = [2,2,2,1,2,4,5,1,3,2,2,2,1]

for i in a_tags:
    
    if 'Category' in i['href']:
        cat_refs.append(BASE+i['href'])
        cats_names.append(i.text)
        count+=1
        if count == 13:
            break
print(cat_refs)

In [ ]:
BASE2 =  "https://www.wicourts.gov/forms1/circuit/"
n = len(cat_refs)

for ref in range(n):
    time.sleep(3)
    soup = connect_to_page(cat_refs[ref])
    a_subcats = soup.findAll('a', href=True)
    for a in a_subcats:
        if 'SubCat=All' in a['href']:
            cat_refs[ref] = BASE2+a['href']
            print(a['href'])
            break
        

print(cat_refs)

In [ ]:
ref_list = ['https://www.wicourts.gov/forms1/circuit/ccform.jsp?page={}&FormName=&FormNumber=&beg_date=&end_date=&StatuteCite=&Category=16', 'https://www.wicourts.gov/forms1/circuit/ccform.jsp?page={}&FormName=&FormNumber=&beg_date=&end_date=&StatuteCite=&Category=6&SubCat=All', 'https://www.wicourts.gov/forms1/circuit/ccform.jsp?page={}&FormName=&FormNumber=&beg_date=&end_date=&StatuteCite=&Category=8&SubCat=All', 'https://www.wicourts.gov/forms1/circuit/ccform.jsp?page={}&FormName=&FormNumber=&beg_date=&end_date=&StatuteCite=&Category=7', 'https://www.wicourts.gov/forms1/circuit/ccform.jsp?page={}&FormName=&FormNumber=&beg_date=&end_date=&StatuteCite=&Category=12&SubCat=All', 'https://www.wicourts.gov/forms1/circuit/ccform.jsp?page={}&FormName=&FormNumber=&beg_date=&end_date=&StatuteCite=&Category=17&SubCat=All', 'https://www.wicourts.gov/forms1/circuit/ccform.jsp?page={}&FormName=&FormNumber=&beg_date=&end_date=&StatuteCite=&Category=21&SubCat=All', 'https://www.wicourts.gov/forms1/circuit/ccform.jsp?page={}&FormName=&FormNumber=&beg_date=&end_date=&StatuteCite=&Category=24', 'https://www.wicourts.gov/forms1/circuit/ccform.jsp?page={}&FormName=&FormNumber=&beg_date=&end_date=&StatuteCite=&Category=51', 'https://www.wicourts.gov/forms1/circuit/ccform.jsp?page={}&FormName=&FormNumber=&beg_date=&end_date=&StatuteCite=&Category=26&SubCat=All', 'https://www.wicourts.gov/forms1/circuit/ccform.jsp?page={}&FormName=&FormNumber=&beg_date=&end_date=&StatuteCite=&Category=38', 'https://www.wicourts.gov/forms1/circuit/ccform.jsp?page={}&FormName=&FormNumber=&Language=es&beg_date=&end_date=&StatuteCite=&Category=&Format=', 'https://www.wicourts.gov/forms1/appeals.jsp?page={}&Category=59']
print(len(cat_page_count))
print(len(cats_names))

In [ ]:
files_df = pd.DataFrame([],columns=["id","jurisdiction","source","title","group","url","filename","downloaded","language"])
pdfs = {}
jur = ' WI'
form_dest = "All_forms/"
today = date.today().strftime("%Y-%m-%d")

for ref in range(len(cats_names)):

    for pages in range(1,cat_page_count[ref]+1):
        time.sleep(1)
        soup = connect_to_page(ref_list[ref].format(pages))
        print(pages, cats_names[ref])
        for tr in soup.find_all('tr'):
            
            
                url_list = tr.find_all('a',href=True)
                for url in url_list:
                    if url!=None and '.pdf' in url['href']:
                        group = cats_names[ref]
                        filename = tr.find('div',id='ccFormNumber').text.strip()
                        if filename not in pdfs:
                            title = tr.find('div',class_='ccFormName').text.strip()
                            pdfs[filename] = title
                            href = BASE + url['href']
                            language = url.text.strip().split("\n")[1].strip()
                            print(filename,title, href)
                            id = hashme(href)
                            status = download_pdf(href,form_dest,id+".pdf")
                            if status=='error':
                                print("#######################")
                                print('ERROR')
                                print("#######################")
                            else:
                                files_df = files_df.append(pd.DataFrame([[id,jur,source,title,group,href,filename,today,language]],columns=["id","jurisdiction","source","title","group","url","filename","downloaded","language"]))

files_df.to_csv("WI_metaData.csv",index=False, encoding="utf-8")